In [1]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import mpl_finance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from datetime import datetime
from openpyxl import load_workbook
import plotly.graph_objects as go
from plotly import subplots

%run _CrawBase.ipynb
%run _BaseInfo.ipynb

df_info=_baseInfo.copy()

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


【craw_stock】 craw_stock stock_number!!!!!!!!! -->8487


In [2]:
def roc_to_gregorian(roc_date_str):
    # 解析字串，假設格式為 "ROC_YEAR/MM/DD"
    roc_year, month, day = roc_date_str.split('/')
    # 轉換為整數
    roc_year = int(roc_year)
    month = int(month)
    day = int(day)
    
    # ROC 年轉西元年
    gregorian_year = roc_year + 1911
    
    # 返回格式化的字串，或是你可以選擇返回 datetime.date 物件
    return pd.to_datetime(f"{gregorian_year:04d}-{month:02d}-{day:02d}")

In [3]:
def save_data_by_date(df, output_dir):
    # 清理无效字符（如 '*'）
    df['日期'] = df['日期'].astype(str).str.replace('*', '', regex=False)

    # 转换为日期格式
    df['日期'] = pd.to_datetime(df['日期'], format='%Y/%m/%d', errors='coerce')

    # 创建存储结果的文件夹
    os.makedirs(output_dir, exist_ok=True)

    # 按日期分组并保存为独立的 Excel 文件
    for date, group in df.groupby('日期'):
        print(date)
        print(roc_to_gregorian(min(end_time_list)))
        if(date>=roc_to_gregorian(min(end_time_list))):
            formatted_date = date.strftime('%Y-%m-%d')  # 格式化日期为文件名
            file_name = f"{output_dir}/{formatted_date}.xlsx"

            # 保存每个分组为单独的 Excel 文件
            group.to_excel(file_name, index=False)

In [4]:
# 計算 RSI 指標
def calculate_rsi(data, window=14):
    delta = data['收盤價'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [5]:
def get_interval(value_to_check, data):
    # 转换数据为浮点数并去除负值和零值
    #data = [float(num) for num in data if float(num) > 0]
    cleaned_numbers = []
    for num in data:
        try:
            value = float(num)
            if value > 0:
                cleaned_numbers.append(value)
        except ValueError:
            # 忽略无法转换的字符串
            continue
    data= cleaned_numbers  
    #print(value_to_check)
    value_to_check=float(value_to_check)
    
    # 计算均值和标准差
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)  # 使用 ddof=1 计算样本标准差

    confidence_levels = [0.40,0.50,0.65, 0.90, 0.95, 0.99]
    
    best_interval = None
    for level in confidence_levels:
        z_score = stats.norm.ppf(1 - (1 - level) / 2)  # 获取 z 分数
        margin_of_error = z_score * std_dev
        
        lower_bound = mean - margin_of_error
        upper_bound = mean + margin_of_error
        # 检查 value_to_check 是否在当前置信区间内
        if lower_bound <= value_to_check <= upper_bound:
            best_interval = (level, lower_bound, upper_bound)
            break  # 如果找到包含 value_to_check 的置信区间，直接退出
    if best_interval:
        level, lower_bound, upper_bound = best_interval
        if mean <value_to_check:
            interval_type = "正區間"
        else:
            interval_type = "負區間"

        #print(f"数据点 {value_to_check} 落在置信水平 {level * 100}% 的{interval_type}: [{lower_bound:.2f}, {upper_bound:.2f}]")
        return (level,interval_type,lower_bound, upper_bound)
    else:
        if mean <value_to_check:
            interval_type = "正區間"
        else:
            interval_type = "負區間"
        
        return (1,interval_type,value_to_check, value_to_check)
        return None

In [6]:
# 清理函式，只對字串型態欄位進行處理
def clean_column(column):
    if column.dtype == 'object':
        return column.str.replace(',', '').replace('0', np.nan).replace('--', '').apply(pd.to_numeric, errors='coerce')
    return column  # 如果已經是數值型態，直接回傳

### Main 


In [7]:
def data_process(RowData_df_craw_stock):

    # 讀取並處理基本資料格式
    df = RowData_df_craw_stock.copy()
    df = df.drop_duplicates()
    
    # 分割日期和年份，將年份轉換為民國年，並重新組合日期
   # df['Year'] = df['日期'].str.split('/', 1).str[0].astype(int) + 1911
   # df['Date'] = df['日期'].str.split('/', 1).str[1]
   # df['日期'] = df.apply(lambda x: f"{x['Year']}/{x['Date']}", axis=1)
   # df['Date_'] = df['日期']
   # df['日期'] = df['日期'].str.replace("＊", "")
    
    df['日期'] = (df['日期'].str.replace("＊", "", regex=False)
                .str.extract(r'(\d{2,3})/(\d{1,2}/\d{1,2})')
                .apply(lambda x: f"{int(x[0]) + 1911}/{x[1]}", axis=1)
    )
   
    # 複製數據
    stock_data = df.copy()
    
    # 轉換數據格式 -確認這些欄位是否需要清理
    columns_to_clean = ['成交金額', '收盤價', '開盤價', '最低價', '最高價', '成交股數', '漲跌價差', '成交筆數']

    # 檢查並清理
    for col in columns_to_clean:
        if col in stock_data.columns:
            try:
                stock_data[col] = clean_column(stock_data[col])
            except Exception as e:
                print(f"Error processing column {col}: {e}")

    stock_data['年月日']=stock_data['日期']

    stock_data['收盤價'] = pd.to_numeric(stock_data['收盤價'], errors='coerce')  # 確保是數字
    stock_data['收盤價'] = stock_data['收盤價'].fillna(method='ffill')  # 前向填補
    
    
    # 計算 EMA (指數移動平均線) -- 使用 12 日的收盤價計算快速 EMA
    stock_data['EMA_12'] = stock_data['收盤價'].ewm(span=12, adjust=False).mean()
    #                         -- 使用 26 日的收盤價計算慢速 EMA
    stock_data['EMA_26'] = stock_data['收盤價'].ewm(span=26, adjust=False).mean()

    # 計算 MACD (移動平均收斂散度) -- MACD 線 = 快速 EMA - 慢速 EMA
    stock_data['MACD'] = stock_data['EMA_12'] - stock_data['EMA_26']
    # 計算 Signal 線 (信號線)     -- 使用 9 日的 MACD 值計算信號線
    stock_data['MACD-SL'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()
    # 計算 DIF (差離值)  --        DIF = MACD 線 - 信號線
    stock_data['DIF']= stock_data['MACD'] - stock_data['MACD-SL']
    # 計算 MACD 黃金交叉       -- 當 MACD 線由下而上穿過信號線時，標記為黃金交叉
    stock_data['MACD_golden_cross'] = (stock_data['MACD'] > stock_data['MACD-SL']) & (stock_data['MACD'].shift(1) <= stock_data['MACD-SL'].shift(1))

    
    # 計算 KD 指標  
    # 計算過去 9 天的最低價
    stock_data['Lowest_Low'] = stock_data['收盤價'].rolling(window=9).min()
    # 計算過去 9 天的最高價
    stock_data['Highest_High'] = stock_data['收盤價'].rolling(window=9).max()
    # 計算 %K 線
    stock_data['%K'] = (stock_data['收盤價'] - stock_data['Lowest_Low']) / (stock_data['Highest_High'] - stock_data['Lowest_Low']) * 100
    # 計算 %D 線，為 %K 線的 3 日移動平均
    stock_data['%D'] = stock_data['%K'].rolling(window=3).mean()

    # 計算 KD 黃金交叉 --  當 %K 線由下而上穿過 %D 線時，標記為黃金交叉
    stock_data['KD_golden_cross'] = (stock_data['%K'] > stock_data['%D']) & (stock_data['%K'].shift(1) <= stock_data['%D'].shift(1))

    
    # 計算成交量相關指標 
    # 計算短期成交量移動平均線（20天）
    stock_data['Volume_MA_short'] = stock_data['成交金額'].rolling(window=5).mean()
    # 計算長期成交量移動平均線（50天）
    stock_data['Volume_MA_long'] = stock_data['成交金額'].rolling(window=10).mean()
    # 計算成交量震盪指標
    stock_data['Volume_Oscillator'] = ((stock_data['Volume_MA_short'] - stock_data['Volume_MA_long']) / stock_data['Volume_MA_long']) * 100
    

    # 計算移動平均線
    # 計算短期移動平均線（20天）
    stock_data['MA_short'] = stock_data['收盤價'].rolling(window=5).mean()
    # 計算長期移動平均線（50天）
    stock_data['MA_long'] = stock_data['收盤價'].rolling(window=20).mean()
    
    
    stock_data['MA_break']= (stock_data['MA_short'] > stock_data['MA_long'] 
                            ) & (
                             stock_data['MA_short'].shift(3) <= stock_data['MA_long'].shift(3)
                            )

    # 計算均線的波動率
    # 計算短期移動平均線的波動率（10天標準差）
    stock_data['MA_short_volatility'] = stock_data['MA_short'].rolling(window=10).std()
    # 計算長期移動平均線的波動率（10天標準差）
    stock_data['MA_long_volatility'] = stock_data['MA_long'].rolling(window=10).std()
    # 當短期移動平均線的波動率大於長期移動平均線的波動率時，標記為糾纏狀態
    stock_data['Is_Entangled'] = stock_data['MA_short_volatility'] > stock_data['MA_long_volatility']

    # 計算均線的變化率
    # 計算短期移動平均線的變化率
    stock_data['MA_short_change'] = stock_data['MA_short'].diff()
    # 計算長期移動平均線的變化率
    stock_data['MA_long_change'] = stock_data['MA_long'].diff()


    # 計算布林帶
    window = 20  # 計算移動平均線和標準差的窗口大小
    num_std_dev = 2  # 標準差倍數，通常設置為2
    # 計算布林帶的中線（移動平均線）
    stock_data['Bollinger_MA'] = stock_data['收盤價'].rolling(window=window).mean()
    # 計算布林帶的標準差
    stock_data['Bollinger_STD'] = stock_data['收盤價'].rolling(window=window).std()
    # 計算布林帶的上限
    stock_data['Bollinger_upper'] = stock_data['Bollinger_MA'] + (num_std_dev * stock_data['Bollinger_STD'])
    # 計算布林帶的下限
    stock_data['Bollinger_lower'] = stock_data['Bollinger_MA'] - (num_std_dev * stock_data['Bollinger_STD'])
    # 當收盤價高於布林帶上限時，標記為布林帶突破
    stock_data['Bollinger_breakout'] = stock_data['收盤價'] > stock_data['Bollinger_upper']

    
    # 計算 RSI
    stock_data['RSI'] = calculate_rsi(stock_data)
    # 當 RSI 從30以下反彈到30以上時，標記為 RSI 反彈
    stock_data['RSI_rebound'] = (stock_data['RSI'] > 30) & (stock_data['RSI'].shift(1) <= 30)

    # 計算漲跌價差和顏色標記
    # 計算收盤價的變化量
    stock_data['Price_Change'] = stock_data['收盤價'].diff()
    
    stock_data['Volume_Price_Change'] = np.where(stock_data['Price_Change'] > 0, 1, -1) * stock_data['成交金額'] 
    stock_data['Volume_Price_Change_sum'] = stock_data['Volume_Price_Change'].rolling(window=9).sum()
    
    stock_data['Volume_Change'] = np.where(stock_data['Price_Change'] > 0, 1, -1) * stock_data['成交股數'] 
    stock_data['Volume_Change_sum'] = stock_data['Volume_Change'].rolling(window=3).sum()
    
    # 根據價差標記顏色，漲為紅色，跌為綠色
    stock_data['Bar_Color'] = stock_data['Price_Change'].apply(lambda x: 'red' if x > 0 else 'green')

    # 計算連續多日收盤價高於移動平均線
    # 當收盤價高於短期移動平均線時，標記為 True
    stock_data['Close_above_MA'] = stock_data['收盤價'] > stock_data['MA_short']
    # 計算連續 5 日收盤價都高於短期移動平均線的天數
    stock_data['Close_above_MA_5days'] = stock_data['Close_above_MA'].rolling(window=5).sum() == 5

    # 計算 Signal_Balance 指標
    # 應用自定義函數 get_analysis_and_count 計算 Analysis 和 Signal_Balance
    stock_data[['Analysis', 'Signal_Balance']] = stock_data.apply(get_analysis_and_count, axis=1, result_type='expand')
    
    
    # 計算 Signal_Balance 的 10 日移動平均線
    stock_data['Signal_Balance_roll'] = stock_data['Signal_Balance'].rolling(window=10).mean()
    # 計算 Signal_Balance 10 日均線的變化量
    stock_data['Signal_Balance_roll_diff'] = stock_data['Signal_Balance_roll'].diff()
    # 當 Signal_Balance 10 日均線上升時，標記為處於上升趨勢
    stock_data['Signal_Balance_uptrend'] = stock_data['Signal_Balance_roll_diff'] > 0
    # 計算連續三日 Signal_Balance 10 日均線上升的天數
    stock_data['Signal_Balance_uptrend_3days'] = stock_data['Signal_Balance_uptrend'].rolling(window=3).sum() == 3

    # 應用趨勢判斷到每一筆資料
    # 使用自定義函數 determine_trend 判斷趨勢
    stock_data['Trend'] = stock_data.apply(determine_trend, axis=1)
    
    # 將趨勢結果加入到 Analysis 中
    stock_data['Analysis'] += stock_data['Trend']

    
    # Calculate and clean up trading volume metrics
    stock_data['短交易量'] = (stock_data['成交金額'] - stock_data['Volume_MA_short']) / stock_data['Volume_MA_short']
    stock_data['遠交易量'] = (stock_data['Volume_MA_short'] - stock_data['Volume_MA_long']) / stock_data['Volume_MA_long']
  
    stock_data['短交易量'] =stock_data['短交易量']* stock_data['Price_Change'].apply(lambda x: 1 if x > 0 else -1)

    
    stock_data['短交易量'] = (stock_data['短交易量'] * 20).fillna('0').astype(int)
    stock_data['短交易量'] = stock_data['短交易量'].rolling(window=5).sum()
    
    stock_data['短增量'] = (abs(stock_data['短交易量'])  / stock_data['短交易量'].shift(1)) ** 0.5
    stock_data['短增量_'] =  stock_data['短增量'].apply(lambda x: x if x > 0 else 0)
    stock_data['短增量_'] =  stock_data['短增量_'].rolling(window=3).mean()
    stock_data['短增量'] = stock_data['短增量'].rolling(window=3).mean()
    
    stock_data['遠交易量'] = (stock_data['遠交易量'] * 100).fillna('0').astype(int)
    
    # # #      # 處理資料
    stock_data['年月日'] = stock_data['年月日'].astype(str).str.replace('*', '', regex=False)
    stock_data['年月日'] = pd.to_datetime(stock_data['年月日'], format='%Y-%m-%d')
    #
    stock_data,merged_intervals= get_highlight(stock_data)
    
    return stock_data

In [8]:
# 判斷每一筆資料的股價趨勢
def determine_trend(row):
    if row['MA_short'] > row['MA_long']:
        if row['MA_short_change'] > 0:
            return '均線:上漲趨勢'
        else:
            return '均線:橫盤震盪'
    elif row['MA_short'] < row['MA_long']:
        if row['MA_short_change'] < 0:
            return '均線:下降趨勢'
        else:
            return '均線:橫盤震盪'
    else:
        return '均線:無明顯趨勢'
    
    
# 數字化判斷每一筆資料的股價趨勢
def determine_trend_level(row):
    if row['MA_short'] > row['MA_long']:
        if row['MA_short_change'] > 0:
            return 4
        else:
            return 2
    elif row['MA_short'] < row['MA_long']:
        if row['MA_short_change'] < 0:
            return 1
        else:
            return 2
    else:
        return 2

In [9]:
def get_analysis_and_count(row):
    
    # 我希望在0~1震盪 所以count加總1
    x=0.25
    
    #计算综合信号平衡值：通过计算买入信号和卖出信号的数量差，得到一个综合的信号值，用于评估当前市场的买卖倾向。
    buy_count = 0
    sell_count = 0
    analysis = ""

    
    # 股價與移動平均線
    if row['收盤價'] > row['MA_short'] and row['MA_short'] > row['MA_long']:
        analysis += "<span style='color:red;'>[股價高於短期均線，短期均線高於長期均線]：買入信號</span><br>"
        buy_count += x
    elif row['收盤價'] < row['MA_short'] and row['MA_short'] < row['MA_long']:
        analysis += "<span style='color:green;'>[股價低於短期均線，短期均線低於長期均線]：賣出信號</span><br>"
        sell_count += x
    
    # 成交金額與成交量震盪指標
    if row['成交金額'] > row['Volume_MA_short'] and row['Volume_Oscillator'] > 0 and row['Price_Change'] >0:
        analysis += "<span style='color:red;'>[成交量大於均線，成交量震盪指標為正]：買入信號</span><br>"
        buy_count += x
    elif row['成交金額'] < row['Volume_MA_short'] and row['Volume_Oscillator'] < 0 and row['Price_Change'] <0:
        analysis += "<span style='color:green;'>[成交量低於均線，成交量震盪指標為負]：賣出信號</span><br>"
        sell_count += x
    
    # KD 指標
    if row['%K'] > row['%D'] and row['%K']<50 and row['%D'] <50 :
        analysis += "<span style='color:red;'>[KD 指標 %K 高於 %D]：買入信號</span><br>"
        buy_count += x
    elif row['%K'] < row['%D'] and row['%K']>80 and row['%D'] >80 :
        analysis += "<span style='color:green;'>[KD 指標 %K 低於 %D]：賣出信號</span><br>"
        sell_count += x
        
        
    #  MACD  
    # 定義成交金額的輔助條件
    volume_condition = row['成交金額'] > row['Volume_MA_short']

    # 結合 RSI 指標進行強買入信號的判斷
    rsi_condition = row['RSI'] < 30  # RSI 指標低於30，表示超賣狀態，可能存在反彈機會
    if row['MACD'] > row['MACD-SL'] and volume_condition and rsi_condition:
        analysis += "<span style='color:red;'>[MACD 高於信號線 (MACD-SL)，成交量增加，RSI 超賣]：強買入信號</span><br>"
        buy_count += x

    # 結合 KD 指標進行強賣出信號的判斷
    kd_condition = row['%K'] > row['%D']  # KD 指標中 %K 線高於 %D 線，表明短期動能強於長期動能
    if row['MACD'] < row['MACD-SL'] and volume_condition and kd_condition:
        analysis += "<span style='color:green;'>[MACD 低於信號線 (MACD-SL)，成交量增加，KD 指標]：強賣出信號</span><br>"
        sell_count += x
    
    return analysis, buy_count# - sell_count

In [10]:
def save_plt_to_html(stock_number,stock_data,fig,text_area):
    try:
        directory = f"Html"    #---本地路徑
        _filename=GetStockInfoByID(stock_number).replace('*', '')
        #print(_filename)
        #檔案路徑設定
        _FilePath = f"{directory}/[{stock_number}]{_filename}.html"  #---本地路徑

        #確認資料夾是否已存在
        if not os.path.exists(directory):
            os.makedirs(directory)

        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++html處理額外新增
        #先存下html在取出，針對head 寫入
        # 需先存下html
        fig.write_html(_FilePath)

        # 读取现有的 HTML 文件
        with open(_FilePath, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

        # html insert 
        head = soup.find('head')
        head.append(BeautifulSoup(text_area, "html.parser"))
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++完成html內容部分

        ## 保存修改后的 HTML 文件 所有的股票都會先存下
        with open(_FilePath, "w", encoding="utf-8") as file:
            file.write(str(soup))

        lastdata=stock_data.iloc[-1]
        
    except Exception as error:
        print(f'Error save_plt_to_html processing  {stock_number}: {error}')

In [11]:
# 找到所有 Upper -> Down 的時間段
def find_intervals(upper, down):
    intervals = []
    for u in upper:
        for d in down:     
            if d > u:
                intervals.append([u, d])
                break
    return intervals

# 合併重維時間段，同時處理重複的標記
def merge_intervals(intervals):
    if not intervals:
        return intervals
    
    # 按每個區間的开始日期排序
    intervals.sort(key=lambda x: x[0])
    merged = []

    for current in intervals:
        if not merged:
            merged.append(current)
        else:
            last = merged[-1]
            if current[0] <= last[1] and current[1] > last[1]:
                # 分割出重複部分，並新增一個新區間
                merged.append([current[0], last[1]])
                merged.append([last[1], current[1]])
            elif current[0] > last[1]:
                merged.append(current)
            else:
                # 更新最後一個區間的結束日期
                last[1] = max(last[1], current[1])

    return merged

In [12]:
def get_highlight(stock_data):
    stock_data['highlight']=False
     
    macd_death_crosses = stock_data[
            (stock_data['MACD'] < stock_data['MACD-SL']) &  # 當前 MACD 線低於信號線
            (stock_data['MACD'].shift(1) >= stock_data['MACD-SL'].shift(1))  # 前一日 MACD 線高於或等於信號線
        ]
    #
    key_area = stock_data[
        (stock_data['MA_short'] > stock_data['MA_long']) &
        (stock_data['%K'] > stock_data['%D']) &
        (stock_data['MACD'] > stock_data['MACD-SL']) &  
        (stock_data['遠交易量'].shift(3) <stock_data['遠交易量'].shift(1)) &  
        (stock_data['遠交易量'] > -25)& 
        (stock_data['短交易量'] > -0)
    ]
    #
    Upper= list(key_area['年月日'])
    Down =list(macd_death_crosses['年月日'])

    Upper.sort()
    Down.sort()
    
    Down.append(pd.Timestamp(datetime.today().date() + timedelta(days=1)))
    intervals = find_intervals(Upper, Down)
    merged_intervals = merge_intervals(intervals)
 
    temp_dates =stock_data['年月日'] 
    
    # 遍历 merged_intervals 进行筛选
    for start, end in merged_intervals:
        mask = (temp_dates >= start) & (temp_dates <= end)
        stock_data.loc[mask, 'highlight'] = True
        stock_data.loc[mask, 'highlight_date'] = start
        stock_data.loc[mask, 'highlight_enddate'] = end

    return stock_data,merged_intervals

In [13]:
def gen_html(stock_number,stock_data):
    
    # 計算 MACD 黃金交叉和死亡交叉
    # 黃金交叉：當前 MACD 線由下而上穿過信號線
    macd_golden_crosses = stock_data[
        (stock_data['MACD'] > stock_data['MACD-SL']) &  # 當前 MACD 線高於信號線
        (stock_data['MACD'].shift(1) <= stock_data['MACD-SL'].shift(1))  # 前一日 MACD 線低於或等於信號線
    ]
 
    # 死亡交叉：當前 MACD 線由上而下穿過信號線
    macd_death_crosses = stock_data[
        (stock_data['MACD'] < stock_data['MACD-SL']) &  # 當前 MACD 線低於信號線
        (stock_data['MACD'].shift(1) >= stock_data['MACD-SL'].shift(1))  # 前一日 MACD 線高於或等於信號線
    ]
 
    # MACD 上升趨勢
    macd_golden_crosses_area  = stock_data[
      
        (stock_data['MACD'] > stock_data['MACD-SL']) &
        (stock_data['MACD'].shift(1) >= stock_data['MACD-SL'].shift(1))
    ]
    
    #交易量 上升趨勢
    Volume_golden_crosses = stock_data[
        (stock_data['遠交易量'].shift(3) <stock_data['遠交易量'].shift(1)) &  
        (stock_data['遠交易量'] > -25)& 
        (stock_data['短交易量'] > -0)
    ]
    
    #
    key_area = stock_data[
        (stock_data['MA_short'] > stock_data['MA_long']) &
        (stock_data['%K'] > stock_data['%D']) &
        (stock_data['MACD'] > stock_data['MACD-SL']) &  
        (stock_data['遠交易量'].shift(3) <stock_data['遠交易量'].shift(1)) &  
        (stock_data['遠交易量'] > -25)& 
        (stock_data['短交易量'] > -0)
    ]
   
    stock_data,merged_intervals=get_highlight(stock_data)
       
    ## 計算 KD 黃金交叉和死亡交叉
    ## 黃金交叉：當前 %K 線由下而上穿過 %D 線
    #kd_golden_crosses = stock_data[
    #    (stock_data['%K'] > stock_data['%D']) &  # 當前 %K 線高於 %D 線
    #    (stock_data['%K'].shift(1) <= stock_data['%D'].shift(1))  # 前一日 %K 線低於或等於 %D 線
    #]

    # 死亡交叉：當前 %K 線由上而下穿過 %D 線
    #kd_death_crosses = stock_data[
    #    (stock_data['%K'] < stock_data['%D']) &  # 當前 %K 線低於 %D 線
    #    (stock_data['%K'].shift(1) >= stock_data['%D'].shift(1))  # 前一日 %K 線高於或等於 %D 線
    #]

    # 計算 RSI 的買入點－找出超賣區回升的點
    # 當 %K 線從 20 以上跌破 20，且 %D 線也在 20 以下時，標記為 RSI 買入點
    rsi_buy_points = stock_data[
        (stock_data['%K'] < 20) &  # 當前 %K 線低於 20
        (stock_data['%K'].shift(1) >= 20) &  # 前一日 %K 線高於或等於 20
        (stock_data['%D'] < 20)  # 當前 %D 線低於 20
    ]

    # 找出超買區回落的點
    # 當 %K 線從 80 以下上升到 80 以上，且 %D 線也在 80 以上時，標記為 RSI 賣出點
    rsi_sell_points = stock_data[
        (stock_data['%K'] > 80) &  # 當前 %K 線高於 80
        (stock_data['%K'].shift(1) <= 80) &  # 前一日 %K 線低於或等於 80
        (stock_data['%D'] > 80)  # 當前 %D 線高於 80
    ]

    # 找出適合買入的點（成交量震盪指標大於 0）
    # 當前成交量震盪指標大於 0，且前一日成交量震盪指標小於或等於 0，標記為買入點
    buy_points = stock_data[
        (stock_data['Volume_Oscillator'] > 0) &  # 當前成交量震盪指標大於 0
        (stock_data['Volume_Oscillator'].shift(1) <= 0)  # 前一日成交量震盪指標小於或等於 0
    ]



    # 創建圖表
    fig = subplots.make_subplots(rows=4, cols=1, 
                        subplot_titles=('股價與移動平均線'#, 'KD 指標'
                                        , 'MACD 指標(有價)','交易量(有量)', '成交金額與成交量震盪指標'),
                        shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        specs=[[{"secondary_y": True}],[{"secondary_y": True}] , [{"secondary_y": True}] ,[{"secondary_y": True}]],
                        row_heights=[0.6, 0.1, 0.1, 0.1]  # 调整各行的高度比例
                        )
    # 添加股票箱型圖到第一圖
    fig.add_trace(go.Candlestick(x=stock_data['年月日'],
                             open=stock_data['開盤價'],
                             high=stock_data['最高價'],
                             low=stock_data['最低價'],
                             close=stock_data['收盤價'],
                             name='箱型圖',
                             increasing_line_color='red', 
                             decreasing_line_color='green',
                             increasing_fillcolor='rgba(255,0,0,0.3)',
                             decreasing_fillcolor='rgba(0,255,0,0.3)'), row=1, col=1)
    #### 圖 #####
    # 添加股價、短期和長期移動平均線、黃金交叉和死亡交叉到第一圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['收盤價'],                  
                             mode='lines', line_color='grey',
                             text=stock_data['Trend'],name='股價'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_short'],                  
                             mode='lines', line_color='blue', 
                             name='壓力-短期移動平均線 (MA 20)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_long'],                  
                             mode='lines', line_color='red', 
                             name='支撐-長期移動平均線 (MA 50)'), row=1, col=1)
    

    fig.add_trace(go.Scatter(x=stock_data[stock_data['Signal_Balance_uptrend_3days']]['年月日'],
                             y=stock_data[stock_data['Signal_Balance_uptrend_3days']]['收盤價'],
                             mode='markers', marker_symbol="triangle-up", marker_color="gold", marker_size=10,
                             text=stock_data['Analysis'],
                             name='觀察點-連續三日上升短期強勢信號'), row=1, col=1, secondary_y=False)
    

    fig.add_trace(go.Scatter(x=stock_data[stock_data['MA_break']]['年月日'],
                         y=stock_data[stock_data['MA_break']]['MA_long'],
                         mode='markers', marker_symbol="star", 
                         marker_color="red", marker_size=10,
                         name='MA_break'), row=1, col=1, secondary_y=False)
    
 
    # 添加成交金額、短期和長期移動平均線、成交量震盪指標以及買入點到第二圖
    fig.add_trace(go.Bar(x=stock_data['年月日'],
                         y=stock_data['成交金額'],
                         name='成交金額',
                         marker_color=stock_data['Bar_Color'],
                         opacity=0.5), row=1, col=1, secondary_y=True)
    
    
    
  
    i_row=1
    # 迴圈遍歷每個區間，將它們加入到圖形中
    for interval in merged_intervals:
      
        x0, x1 = interval
        fig.add_vrect(x0=x0, x1=x1,
                      annotation_text="--", annotation_position="top left",
                      fillcolor="#f6b26b", opacity=0.25, line_width=0, row=i_row, col=1)

    
    #fig.update_yaxes(title_text="成交金額", row=i_row, col=1)
    #fig.update_yaxes(title_text="成交量震盪指標 (VO)", row=i_row, col=1, secondary_y=True, range=[stock_data['Volume_Oscillator'].min(), stock_data['Volume_Oscillator'].max()])


    # 添加 KD 指標和黃金交叉到第三圖
#    fig.add_trace(go.Scatter(x=stock_data['年月日'],
#                             y=stock_data['%K'],
#                             mode='lines', line_color='#ebbd67',
#                             name='%K(9)'), row=3, col=1)
#    fig.add_trace(go.Scatter(x=stock_data['年月日'],
#                             y=stock_data['%D'],
#                             mode='lines', line_color='#67ceeb',
#                             name='%D(3)'), row=3, col=1)
#    fig.add_trace(go.Scatter(x=rsi_buy_points ['年月日'], 
#                             y=rsi_buy_points ['%K'], 
#                             mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=10,
#                             name='超賣區回升'), row=3, col=1)
#    fig.add_trace(go.Scatter(x=rsi_sell_points ['年月日'], 
#                             y=rsi_sell_points ['%K'], 
#                             mode='markers', marker_symbol="triangle-down", marker_color="black", marker_size=10,
#                             name='超買區回落'), row=3, col=1)
    
    i_row=2

    # 添加 MACD 指標、Signal Line、DIF 以及 MACD 的黃金交叉和死亡交叉到第四圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MACD'],
                             mode='lines', line_color='#ebbd67',
                             name='Diff(12,26)'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MACD-SL'],
                             mode='lines', line_color='#67ceeb',
                             name='MACD(9)'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=macd_golden_crosses['年月日'], 
                             y=macd_golden_crosses['MACD'], 
                             mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=10,
                             name='MACD 黃金交叉'), row=i_row, col=1)
    fig.add_trace(go.Scatter(x=macd_death_crosses['年月日'], 
                             y=macd_death_crosses['MACD'], 
                             mode='markers', marker_symbol="triangle-down", marker_color="black", marker_size=10,
                             name='MACD 死亡交叉'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'], 
                             y=stock_data['Volume_Price_Change_sum'], 
                              mode='lines', line_color='#67ceeb',
                             name='Price_Change'), row=i_row, col=1, secondary_y=True)
    #Price_Change Volume_Price_Change_sum
    fig.add_trace(go.Scatter(x=macd_golden_crosses_area['年月日'],
                             y=macd_golden_crosses_area['MACD'],
                             mode='markers', line_color='red',
                             name='Macd上升'), row=i_row, col=1)
    

    fig.update_yaxes(title_text="MACD", row=i_row, col=1)
    
    # 
    i_row=3
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['短交易量'],
                             mode='lines', line_color='#ebbd67',
                             name='短交易量'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['遠交易量'],
                             mode='lines', line_color='#67ceeb',
                             name='遠交易量'), row=i_row, col=1)
    
    fig.add_trace(go.Scatter(x=Volume_golden_crosses['年月日'],
                             y=Volume_golden_crosses['遠交易量'],
                             mode='markers', line_color='red',
                             name='交易上升'), row=i_row, col=1)
    
    #fig.add_trace(go.Scatter(x=stock_data['年月日'],
    #                         y=stock_data['短增量_'],
    #                         mode='lines', line_color='#e06666',
    #                         name='短增量'), row=i_row, col=1, secondary_y=True)
    
    fig.update_yaxes(title_text="交易量", row=i_row, col=1)
    
    
    
      
    #### 圖 #####
    i_row=4
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Volume_MA_short'],
                             mode='lines', line_color='#ebbd67',
                             name='成交金額短期移動平均線 (MA 20)'), row=i_row, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Volume_MA_long'],
                             mode='lines', line_color='#67ceeb',
                             name='成交金額長期移動平均線 (MA 50)'), row=i_row, col=1 )
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Volume_Oscillator'],
                             mode='lines', line_color='gray',
                             name='成交量震盪指標 (VO)'), row=i_row, col=1, secondary_y=True)
    
    fig.add_trace(go.Scatter(x=buy_points['年月日'],
                             y=buy_points['Volume_MA_short'],
                             mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=10,
                             name='成交量增加'), row=i_row, col=1)
    
    
    
    fig.add_trace(go.Scatter(x=key_area['年月日'],
                             y=key_area['Volume_MA_long'],
                             mode='markers', marker_symbol="star",  line_color='red',
                             name='中'), row=i_row, col=1) 
    

    # 設置 x 和 y 軸
    fig.update_yaxes(title_text="股價", row=1, col=1)
    fig.update_xaxes(title_text="日期", row=4, col=1)
  

    #    fig.update_yaxes(title_text="KD 指標 (%)", row=3, col=1)
   
   
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)

    ########################
    #  想在圖片右下角備註一些指標情況
    ########################
    
    # 定义灯号和文字的位置、颜色
    transparent_color = 'gray'
    indicator_positions = [1, 2, 3, 4]
    indicator_labels = [
        'MACD 黃金交叉',
        'KD 黃金交叉',
        '觀察點-連續三日上升短期強勢信號',
        '轉強點-加權市場信號'
    ]
    indicator_colors = [
        'red' if stock_data['KD_golden_cross'].iloc[-1] else transparent_color,
        'red' if stock_data['Signal_Balance_uptrend_3days'].iloc[-1] else transparent_color,
        #'red' if stock_data['Weighted_Signa_over_threshold'].iloc[-1] else transparent_color
    ]

    # 生成 HTML 内容
    html_content = ""
    for label, color in zip(indicator_labels, indicator_colors):
        html_content += f"""
        <div style="display: flex; align-items: center;">
            <div style="width: 10px; height: 10px; border-radius: 50%; background-color: {color}; margin-right: 8px;"></div>
            <span>{label}</span>
        </div><br>
    """
        
    html_iframe = f"""
    <div style="width: 100%; height: 250%; overflow: auto;">
      
        <div id="iframe-container" style="height:750px;display: none;">
            <iframe src="https://www.wantgoo.com/stock/{stock_number}" width="100%" height="100%" frameborder="0"></iframe>
        </div>
        <button onclick="toggleIframe()">顯示/隱藏 iframe</button>
        <a href="https://www.wantgoo.com/stock/{stock_number}"  target="_blank">玩股網</a>
        <a href=" https://tw.stock.yahoo.com/quote/{stock_number}.TW"  target="_blank">yahoo</a>
        <a href=" https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a={stock_number}"  target="_blank">moneydj</a>
    </div>
    <script>
        function toggleIframe() {{
            var iframeContainer = document.getElementById('iframe-container');
            if (iframeContainer.style.display === 'none') {{
                iframeContainer.style.display = 'block';
            }} else {{
                iframeContainer.style.display = 'none';
            }}
        }}
    </script>
    
    """

  
    
    # 添加 HTML 内容到图表
    html_text =html_iframe + '<br>'+ html_content+ '<br>'+ stock_data['Analysis'].iloc[-1]
    # MARK CONTENT 
    html_text =html_iframe 
    # 创建新的文本区域并使用 CSS 定位到右下角
    text_area = f'''
    <style>
        .text-area {{
            position: fixed;
            bottom: 10px;
            right: 10px;
            background-color: white;
            padding: 10px;
            border: 1px solid black;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
            z-index: 1000;
            max-width: 300px;
            word-wrap: break-word;
        }}
    </style>
    <div class="text-area">
        {html_text}
    </div>
    '''

    save_plt_to_html(stock_number,stock_data,fig,text_area)

In [14]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import threading

In [15]:
global df_save_df_to_excel
global df_save_old_RunRealTimeStock
df_save_df_to_excel=pd.DataFrame()
df_save_old_RunRealTimeStock=pd.DataFrame()

def save_to_df(_df):
    _df['年月日'] = _df['年月日'].dt.strftime('%Y-%m-%d')
    
    # 處理資料 df1
    df1=_df
    
    # 處理資料 df2
    _stock_data=_df
    to_RunRealTimeStock_excel=_df
    to_RunRealTimeStock_excel['now_price']=_stock_data['收盤價']
    to_RunRealTimeStock_excel['change_price']=_stock_data['漲跌價差']
    to_RunRealTimeStock_excel['change_quote'] = (_stock_data['漲跌價差'].astype(float, errors='ignore') / _stock_data['開盤價'].astype(float, errors='ignore')).replace([float('inf'), -float('inf'), float('nan')], 0) * 100

    to_RunRealTimeStock_excel['change_quote'] = to_RunRealTimeStock_excel['change_quote'].astype(float).apply(lambda x: f"{x:.2f}%")
    df2=to_RunRealTimeStock_excel[['日期','stock_number','now_price','change_price','change_quote']]

    # 更新全局 DataFrame
    global df_save_df_to_excel  
    if  df_save_df_to_excel.empty:
        df_save_df_to_excel=df1
    else :
        df_save_df_to_excel = pd.concat([df_save_df_to_excel, df1], ignore_index=True)
       
    global df_save_old_RunRealTimeStock  
    if  df_save_old_RunRealTimeStock.empty:
        df_save_old_RunRealTimeStock=df2
    else :
        df_save_old_RunRealTimeStock = pd.concat([df_save_old_RunRealTimeStock, df2], ignore_index=True)

In [16]:

# 定义保存 Excel 文件的函数
def save_process_data(stock_number, stock_data):
    try:
        if stock_data.empty:
            print(f'DataFrame is empty. Skipping save.{stock_number}')
            return
        
        level, interval_type, lower_bound, upper_bound = get_interval(stock_data.iloc[-1]['收盤價'], np.array(stock_data['收盤價']))
        stock_data['stock_number'] = stock_number
        stock_data['level'] = level
        stock_data['interval_type'] = interval_type
        stock_data['lower_bound'] = lower_bound
        stock_data['upper_bound'] = upper_bound
        
        #存excel 
        _max_end_date=max(end_time_list)
        _min_end_date=min(end_time_list) 
        max_end_date = f"{int(_max_end_date.split('/')[0]) + 1911}-{_max_end_date.split('/')[1]}-{_max_end_date.split('/')[2]}"
        min_end_date=f"{int(_min_end_date.split('/')[0]) + 1911}-{_min_end_date.split('/')[1]}-{_min_end_date.split('/')[2]}"

        stock_data = stock_data[
                    (stock_data['年月日'] <= max_end_date) & 
                    (stock_data['年月日'] >= min_end_date)
        ]
        save_to_df(stock_data)
        
    except Exception as error:
        print(f'Error in save_process_data for stock {stock_number}: {error}')

# 定义主要爬虫和处理函数
def process_stock_codes(stock_number):
    try:
        ## Test
        # craw_stock_need_update=Truw
        #global _dummyData
        
        craw_stock_need_update=False
        RowData_df_craw_stock, His_Stock, isSuccess = craw_stock(stock_number, start_month,datetime.now().strftime("%Y-%m-%d"),craw_stock_need_update)
        _dummyData=data_process(RowData_df_craw_stock)

        #存html 
        gen_html(stock_number, _dummyData)  
        save_process_data( stock_number, _dummyData)

    except Exception as error:
        print(f'Error processing stock {stock_number}: {error}')

# 分別處理不同的股票列表
def process_stock_list(stock_list):
    for stock_number in stock_list:
        process_stock_codes(stock_number)

In [17]:
## 【Run】 跑數據
start_month = '2023-09-01'

start_date = datetime.strptime(start_month, "%Y-%m-%d").date()
today = datetime.today().date()

def to_roc(date_obj):
    roc_year = date_obj.year - 1911
    return f"{roc_year}/{date_obj.month:02d}/{date_obj.day:02d}"

end_time_list = []
current_date = start_date
while current_date <= today:
    end_time_list.append(to_roc(current_date))
    current_date += timedelta(days=1)

end_time_list=end_time_list[-10:]

min(end_time_list) 


'114/05/04'

process_stock_codes('2365')

In [20]:
#process_stock_codes('6169')
process_stock_codes('1101')

【craw_stock】 craw_stock stock_number!!!!!!!!! -->1101


In [19]:
# Test
#_dummyData[_dummyData['日期']>'2025/03/10']

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,年月日,EMA_12,EMA_26,MACD,MACD-SL,DIF,MACD_golden_cross,Lowest_Low,Highest_High,%K,%D,KD_golden_cross,Volume_MA_short,Volume_MA_long,Volume_Oscillator,MA_short,MA_long,MA_break,MA_short_volatility,MA_long_volatility,Is_Entangled,MA_short_change,MA_long_change,Bollinger_MA,Bollinger_STD,Bollinger_upper,Bollinger_lower,Bollinger_breakout,RSI,RSI_rebound,Price_Change,Volume_Price_Change,Volume_Price_Change_sum,Volume_Change,Volume_Change_sum,Bar_Color,Close_above_MA,Close_above_MA_5days,Analysis,Signal_Balance,Signal_Balance_roll,Signal_Balance_roll_diff,Signal_Balance_uptrend,Signal_Balance_uptrend_3days,Trend,短交易量,遠交易量,短增量,短增量_,highlight,highlight_date,highlight_enddate,stock_number,level,interval_type,lower_bound,upper_bound
365,2025/03/11,17,348,20.65,20.65,20.30,20.55,-0.10,73,2025-03-11,20.419366,20.415289,0.004077,-0.039832,0.043909,False,20.30,20.65,71.428571,78.354978,False,268.0,383.3,-30.080877,20.46,20.3850,False,0.058013,0.033612,True,0.05,0.0050,20.3850,0.177779,20.740557,20.029443,False,49.122807,False,-0.10,-348,1559.0,-17,15.0,green,True,False,<span style='color:red;'>[股價高於短期均線，短期均線高於長期均線]...,0.25,0.100,0.000,False,False,均線:上漲趨勢,1.0,-30,0.558509,0.558509,True,2025-03-05,2025-03-13,6169,1,正區間,27.3,27.3
366,2025/03/12,32,651,20.15,20.55,20.10,20.10,-0.45,60,2025-03-12,20.370233,20.391935,-0.021702,-0.036206,0.014504,False,20.10,20.65,0.000000,57.142857,False,385.8,433.8,-11.065007,20.41,20.3650,False,0.059151,0.035575,True,-0.05,-0.0200,20.3650,0.186449,20.737898,19.992102,False,43.750000,False,-0.45,-651,695.0,-32,-40.0,green,False,False,均線:橫盤震盪,0.00,0.100,0.000,False,False,均線:橫盤震盪,5.0,-11,1.068163,1.068163,True,2025-03-05,2025-03-13,6169,1,正區間,27.3,27.3
367,2025/03/13,11,225,20.00,20.55,20.00,20.05,-0.05,64,2025-03-13,20.320966,20.366606,-0.045640,-0.038093,-0.007547,False,20.05,20.65,0.000000,23.809524,False,377.8,435.0,-13.149425,20.35,20.3525,False,0.050563,0.035024,True,-0.06,-0.0125,20.3525,0.198994,20.750488,19.954512,False,43.076923,False,-0.05,-225,-1213.0,-11,-60.0,green,False,False,<span style='color:green;'>[股價低於短期均線，短期均線低於長期均...,0.00,0.100,0.000,False,False,均線:下降趨勢,4.0,-13,1.126832,1.126832,True,2025-03-05,2025-03-13,6169,1,正區間,27.3,27.3
368,2025/03/14,3,62,20.55,20.55,20.10,20.15,0.10,45,2025-03-14,20.294664,20.350561,-0.055897,-0.041654,-0.014244,False,20.05,20.65,16.666667,5.555556,True,294.8,272.9,8.024918,20.30,20.3500,False,0.048944,0.034530,True,-0.05,-0.0025,20.3500,0.201311,20.752623,19.947377,False,52.631579,False,0.10,62,-947.0,3,-40.0,red,False,False,<span style='color:green;'>[股價低於短期均線，短期均線低於長期均...,0.25,0.100,0.000,False,False,均線:下降趨勢,-29.0,8,1.941026,1.941026,False,NaT,NaT,6169,1,正區間,27.3,27.3
369,2025/03/17,14,286,20.15,20.60,20.15,20.15,0.00,77,2025-03-17,20.272408,20.335705,-0.063297,-0.045982,-0.017315,False,20.05,20.65,16.666667,11.111111,False,314.4,281.1,11.846318,20.20,20.3325,False,0.072426,0.033008,True,-0.10,-0.0175,20.3325,0.202793,20.738086,19.926914,False,42.553191,False,0.00,-286,-986.0,-14,-22.0,green,False,False,<span style='color:green;'>[股價低於短期均線，短期均線低於長期均...,0.25,0.125,0.025,True,False,均線:下降趨勢,-24.0,11,NaN,1.195670,False,NaT,NaT,6169,1,正區間,27.3,27.3
370,2025/03/18,19,387,19.75,20.45,19.75,20.00,-0.15,81,2025-03-18,20.230499,20.310838,-0.080339,-0.052854,-0.027485,False,20.00,20.65,0.000000,11.111111,False,322.2,295.1,9.183328,20.09,20.3200,False,0.110398,0.031229,True,-0.11,-0.0125,20.3200,0.215455,20.750911,19.889089,False,48.780488,False,-0.15,-387,-1435.0,-19,-30.0,green,False,False,<span style='color:green;'>[股價低於短期均線，短期均線低於長期均...,0.00,0.125,0.000,False,False,均線:下降趨勢,-23.0,9,NaN,0.897527,False,NaT,NaT,6169,1,正區間,27.3,27.3
371,2025/03/19,10,194,20.20,20.40,20.00,20.40,0.40,68,2025-03-19,20.256576,20.317442,-0.060866,-0.054456,-0.006410,False,20.00,20.65,61.538462,26.068376,True,230.8,308.3,-25.137853,20.15,20.3100,False,0.122968,0.029031,True,0.06,-0.0100,20.3100,0.206219,20.722438,19.897562,False,52.272727,False,0.40,194,-976.0